In [5]:
import pandas as pd
import math
from collections import Counter
from pprint import pprint





In [7]:
data=pd.read_csv('p3.csv')
print(data.head())
attrs=list(data.columns)

attrs.remove('play')
attrs.remove('day')

  day outlook temperature humidity wind play
0                  d1 sunny hot high weak no
1               d2 sunny hot high strong  no
2              d3 overcast hot high weak yes
3                 d4 rain mild high weak yes
4               d5 rain cool normal weak yes


ValueError: list.remove(x): x not in list

In [ ]:
def entropy(probs):
    return sum([-prob*math.log(prob,2) for prob in probs])

def entropy_list(a_list):
    count=Counter(x for x in a_list)
    n= len(a_list)*1.0
    probs=[x/n for x in count.values()]
    return entropy(probs)

total_entropy=entropy_list(data['play'])
def information_gain(data, split_attr, target_attr, trace=1):
    data_split= data.groupby(split_attr)
    obs= len(data.index)*1.0
    print(obs)
    data_agg= data_split.agg({target_attr:[entropy_list,lambda x:len(x)/obs]})
    data_agg.columns= ['Entropy','propobservations']
    new_entropy= sum(data_agg['Entropy']*data_agg['propobservations'])
    old_entropy= entropy_list(data[target_attr])
    return old_entropy-new_entropy

information_gain(data, 'outlook', 'play', trace=1)

def id3(data,target_attr,attr_names,default_class=None):
    cut= Counter(x for x in data[target_attr])
    if len(cut)==1:
        return next(iter(cut))
    elif data.empty or (not attr_names):
        return default_class
    else:
        default_class= max(cut.keys())
        gain= [information_gain(data,attr,target_attr) for attr in attr_names]
        index_max= gain.index(max(gain))
        best_attr= attr_names[index_max]
        tree= {best_attr:{}}
        remaining_attr_names= [i for i in attr_names if i!=best_attr]
        for attr_val,data_subset in data.groupby(best_attr):
            subtree=id3(data_subset,target_attr,remaining_attr_names,default_class)
            tree[best_attr][attr_val]=subtree
    return tree
tree=id3(data,'play',attrs)
print("\n\n The Resultant Decission Tree is:\n")
pprint(tree)